In [1]:
import win32com.client
import pandas as pd
import re
from datetime import datetime, timedelta

In [2]:
your_name = "Sachin"

# Your company domain (for filtering internal emails)
internal_domain = "@spglobal.com"

# Connect to Outlook
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

# Select Inbox (can be modified if you're using a subfolder)
inbox = outlook.GetDefaultFolder(6)  # 6 refers to inbox
messages = inbox.Items
messages.Sort("[ReceivedTime]", True)  # sort by latest first
# cond
# messages = messages.Restrict(f"[ReceivedTime] >= '{yesterday}'")


In [3]:
emails_to_ignore = [
    'Kapow.Sourcing@spglobal.com'
]

In [4]:
pattern = r"@spglobal.com"

data = []
for message in messages:
    try:

        if message.SenderEmailType == "EX":
            # For Exchange senders (internal colleagues)
            smtp_address = message.Sender.GetExchangeUser().PrimarySmtpAddress
        else:
            # For external SMTP senders
            smtp_address = message.SenderEmailAddress

        if (smtp_address is not None) & (smtp_address.endswith(pattern)):
            if smtp_address not in emails_to_ignore:
                subject = message.Subject
                body = message.Body
                date = message.ReceivedTime.strftime("%Y-%m-%d")
                if (date < '2025-06-28'):
                    break
                mentioned = "yes" if re.search(rf"\b{your_name}\b", body, re.IGNORECASE) else "no"
                data.append([date, subject, body, smtp_address, mentioned])
                
    except Exception as e:
        # Skip if something fails (e.g., a meeting invite without a sender email)
        continue
    


In [6]:
# Create DataFrame
df = pd.DataFrame(data, columns=["Date", "subject", "body", "sender", "Mentioned"]).sort_values(by="sender", ascending=False)
df.to_csv('email.csv', index=False)

In [8]:
df_p = df[df.Mentioned == 'yes'][['sender', 'subject', 'body']]
df_p.shape

(5, 3)

## Deploying LLM extraction and summarization tool using Ollama model

In [9]:
! pip install -q langchain langgraph openai pandas Langchain-community langchain-ollama

In [10]:
from langchain_core.runnables import Runnable
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from langchain_community.chat_models import ChatOllama

## 
llm = ChatOllama(model="mistral")

C:\Users\sachin.verma\AppData\Local\Temp\ipykernel_30180\2470696828.py:7: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="mistral")


In [ ]:
result = llm.invoke(
    [HumanMessage(content= """Hi""")]
)

In [12]:
result

AIMessage(content=" Hello! How can I help you today?\n\nAsk me anything related to programming, data science, machine learning, or artificial intelligence, and I'll do my best to assist you. If you have any specific problems or projects you're working on, feel free to share them with me. I'm here to learn and grow alongside you!\n\nIf you have any other topics you'd like to discuss, just let me know. I'm always excited to engage in interesting conversations and help others along their learning journey. What can I assist you with today?", additional_kwargs={}, response_metadata={'model': 'mistral', 'created_at': '2025-07-01T09:34:50.5088602Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 109837304200, 'load_duration': 18104900, 'prompt_eval_count': 5, 'prompt_eval_duration': 50428572500, 'eval_count': 120, 'eval_duration': 59389565700}, id='run--af38b1b6-eccc-4a69-b724-f343f9b61b89-0')

In [13]:
from langchain_community.chat_models import ChatOllama
from langgraph.graph import StateGraph, END
from langchain_core.runnables import Runnable
from IPython.display import display, Markdown

# 1. Preprocess
def preprocess_email(state):
    email = state["email"]
    full_text = f"Sender: {email['sender']}\nSubject: {email['subject']}\nBody: {email['body']}"
    return {"email_text": full_text}

# 2. Summarize using Ollama
def summarize_with_ollama(state):
    prompt = f"""You are an email assistant. Summarize this email and suggest a reply. keep the headings in bold and use markdown format. Dont give any additional information:\n\n{state['email_text']}"""
    response = llm.invoke(prompt)
    return {"response": response.content}

# 3. Output step
def output_summary(state):
    display("AI Summary & Suggested Reply:")
    display(Markdown(state["response"]))
    return state

# 4. Define the graph
graph = StateGraph(dict)
graph.add_node("preprocess", preprocess_email)
graph.add_node("summarize", summarize_with_ollama)
graph.add_node("output", output_summary)

graph.set_entry_point("preprocess")
graph.add_edge("preprocess", "summarize")
graph.add_edge("summarize", "output")
graph.add_edge("output", END)

chain = graph.compile()


## Invoke the Chain


In [14]:
for _, row in df_p.iterrows():
    print("🟢 Processing email...\n")
    state = {"email": row.to_dict()}
    chain.invoke(state)
    print("\n" + "+"*200 + "\n")

🟢 Processing email...



'AI Summary & Suggested Reply:'

 **Email Summary**

The email chain discusses a data extraction issue for the Port Hedland source. Initially, Vismaya reported the problem on May 6th and requested Manoj's team to look into it. On April 21st, Vismaya followed up again, explaining that there were deviations in the data extraction for the latest file of the Port Hedland source. Varsha responded on June 30th, stating that the issue has been resolved and only the correct file will be uploaded to S3 moving forward.

**Suggested Reply:**

Hi Varsha,

Thank you for promptly addressing the data extraction issue for the Port Hedland source. We appreciate your team's quick response and ensure that there are no further issues going forward. If we have any concerns or questions regarding this matter, we will reach out to you immediately.

Best Regards,
[Your Name]


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

🟢 Processing email...



'AI Summary & Suggested Reply:'

...

 It appears that this text contains the transcript of an email or message, which includes a meeting invitation and a request for data extraction. Here's a summary:

1. The meeting is about data extraction. The subject line is not provided in this text snippet.
2. The meeting ID is 254 064 036 465, with the passcode "kLckge". You can join by phone using +1 862-294-2468,,235636981# or find a local number <https://nam11.safelinks.protection.outlook.com/?url=https%3A%2F%2Fdialin.teams.microsoft.com%2Fa6bff586-987b-4390-a787-748bbde14b6d%3Fid%3D235636981&data=05%7C02%7Csachin.verma%40spglobal.com%7C3a52a4d43ca34b54c29a08ddb867d184%7C8f3e36ea80394b4081a77dc0599e8645%7C1%7C0%7C638869478238869727%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C0%7C%7C%7C&sdata=N1OtFHflMsFp1UXpZ69FEejZ0dt%2FT4nzo%2FD3uLWiTao%3D&reserved=0>.
3. The meeting can also be joined on a video conferencing device with the tenant key: 762655058@t.plcm.vc and Video ID: 118 104 348 6.
4. Additional information for organizers, such as meeting options and reset dial-in PIN, can be found at the provided links.
5. The body of the message does not contain any explicit request or instructions regarding data extraction, but it seems that the purpose of the meeting is to discuss the details of this task.


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

🟢 Processing email...



'AI Summary & Suggested Reply:'

 **Summary**

* An email from SPGlobalSPAMQuarantine@spglobal.com regarding a new message in the quarantine for Sachin Verma (sachin.verma@spglobal.com) was received.
* The email contains links to view and manage the digest, safe/blocked senders list, and account.
* One email from events@it-virtual-summits.com titled "AI Deployment Summit - A Virtual Event - Registration Now!" is in the quarantine and can be released or allowed/blocked.

**Reply Suggestion**

Subject: Re: Spam Quarantine Notification: 1 New Message

Dear SPGlobalSPAMQuarantine Team,

Thank you for notifying me about the new message in my quarantine. I have reviewed the information and will take appropriate action regarding the email from events@it-virtual-summits.com.

Best regards,
Sachin Verma


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

